In [1]:
from robot_library_py import *
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy, JointState
from intera_core_msgs.msg import JointCommand

from threading import Thread
import time
import numpy as np 
from std_msgs.msg import Bool
import rotm2euler

In [2]:
#worked with sim.

In [2]:
class ROSInterface:
    def __init__(self, node, robot):
        
        self.robot = robot
        self.node = node
        self.jointMap = {name: ind for ind, name in enumerate(self.robot.jointNames)} 

        self.latest_joint_states = None

    def joint_states_callback(self, msg):
        self.latest_joint_states = msg
        q = self.robot.getJoints()
        # print('joint_states:', q)
        for ind, name in enumerate(msg.name):
            if name in self.jointMap:
                q[self.jointMap[name]] = msg.position[ind]
        self.robot.setJoints(q)

    def spin_thread(self):
        self.node.create_subscription(JointState, 'robot/joint_states', self.joint_states_callback, 10) 
        rclpy.spin(self.node)

In [3]:
rclpy.init()
robot = URDFModel('/home/ns/sawyer_robot_ros2/src/teleop_script/sawyer.urdf')
jointMap = {name: ind for ind, name in enumerate(robot.jointNames)}

node = Node('pub_dbg')
q = robot.getJoints()
ros_interface = ROSInterface(node, robot)
cmd_pub = node.create_publisher(JointCommand, '/robot/limb/right/joint_command', 10)
pass 

sawyer
link: 0
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: base
link: 1
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name: torso
link: 2
	cog:    0    0 -0.5
	inertia:    5.06359    6.08689    4.96192 0.00105311   0.801996 0.00103417
	mass: 60.864
	name: pedestal
link: 3
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: controller_box
link: 4
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: pedestal_feet
link: 5
	cog:  -0.0006241 -2.8025e-05    0.065404
	inertia:   0.0067599   0.0067877   0.0074031  1.5888e-05 -6.1904e-07 -4.2024e-05
	mass: 2.0687
	name: right_arm_base_link
link: 6
	cog: 0.024366 0.010969  0.14363
	inertia:  0.053314  0.057902  0.023659 0.0080179  0.011734 0.0047093
	mass: 5.3213
	name: right_l0
link: 7
	cog:   0.0053207 -2.6549e-05      0.1021
	inertia:    0.011833   0.0082709   0.0049661  4.2124e-07  4.9425e-05 -4.4669e-06
	mass: 1.5795
	name: head
link: 8
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name

In [42]:
robot.EENames

['controller_box',
 'pedestal_feet',
 'pedestal',
 'screen',
 'head_camera',
 'right_torso_itb',
 'right_arm_itb',
 'right_hand_camera',
 'right_wrist',
 'right_gripper_l_finger_tip',
 'right_gripper_r_finger_tip',
 'right_gripper_tip',
 'camera_link',
 'right_l4_2',
 'right_l2_2',
 'right_l1_2',
 'torso']

In [5]:
# ros_interface.spin_thread()
t1 = Thread(target=ros_interface.spin_thread)
t1.start()

In [6]:
ros_interface.robot.bodyNames[18]

'right_l6'

In [7]:
js=ros_interface.latest_joint_states
header=js.header
names=js.name
poss=np.array(js.position)
vels=np.array(js.velocity) 
j_gripper=poss[-2]
gripper_close=np.allclose(0, j_gripper)

gripper_close

True

In [8]:
q=ros_interface.robot.getJoints()
j_gripper=q[-2]
gripper_close=np.allclose(0, j_gripper)

gripper_close

True

In [9]:
cj=ros_interface.robot.getJoints()
cj 

array([-5.76854590e-03, -1.61511168e-01,  3.98745865e-01, -1.75169992e+00,
        1.47252977e+00, -1.13550389e+00, -1.73252583e+00,  3.89936537e-01,
        0.00000000e+00, -1.47179753e-05])

In [14]:
msg=ros_interface.latest_joint_states
poss_prev=np.array(msg.position)

msgs=[]
motion_detected=False
stop_count=0
for i in range(1000000):
    time.sleep(0.5)
    msg=ros_interface.latest_joint_states
    msgs.append(msg)
    poss=np.array(msg.position)
    d=np.linalg.norm(poss-poss_prev)
    if np.allclose(poss_prev, poss):  # no motion
        if motion_detected:
            # print('motion stopped, d=', d)
            stop_count+=1
            if stop_count>1:
                print('stop')
                break
    elif not motion_detected:
        print('motion detected')
        motion_detected=True
        
    poss_prev=poss.copy()


motion detected
stop


In [16]:
len(msgs)

10

### record 10 points with fixed interval

In [18]:
msg=ros_interface.latest_joint_states
poss_prev=np.array(msg.position)

msgs=[]
for i in range(10):
    print('i=', i)
    time.sleep(1)
    msg=ros_interface.latest_joint_states
    msgs.append(msg)
print('stop')


i= 0
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
stop


In [19]:
def get_ee(jointmsg):
    msg=jointmsg    
    q =  robot.getJoints() 
    for ind, name in enumerate(msg.name):
        if name in jointMap:
            q[jointMap[name]] = msg.position[ind]
    robot.setJoints(q)
    
    # T=robot.getBodyTransform('right_l6');
    T=robot.getBodyTransform('camera_link');
    xyz=T[0:3,3]
    e1 = rotm2euler.rotationMatrixToEulerAngles(T[0:3, 0:3])
    ee6=np.concatenate([xyz, e1])
    return ee6

In [43]:
ee6_0=get_ee(msgs[0])
ee6_0

array([ 0.48078743, -0.16970559,  0.4446836 ,  1.38204485,  1.54640339,
        1.30472309])

In [44]:
ee6_n=get_ee(msgs[-1])
ee6_n

array([ 0.68582614, -0.24117659,  0.25773452,  1.57555635,  1.54929995,
        1.49310114])

In [45]:
get_ee(ros_interface.latest_joint_states)

array([ 0.66532176, -0.23861144,  0.28409677,  1.93164498,  1.56222109,
        1.85519242])

array([ 0.48078743, -0.16970559,  0.4446836 ,  1.38204485,  1.54640339,
        1.30472309])

In [92]:
# target=ee6_0.copy() 
target=ee6_n.copy() 

delay=0.1
alpha = .0001
I = np.eye(q.shape[0])

In [93]:
# for i in range(50):
while np.linalg.norm(xd)>0.1:
    ee6=get_ee(ros_interface.latest_joint_states)
    xd=target-ee6 
    xd[3:6]=0

    J = robot.getJacobian('camera_link')
    
    qd = np.linalg.inv(J.T @ J + alpha * I) @ J.T @ xd

    cmd_msg = JointCommand()
    cmd_msg.names = robot.jointNames
    cmd_msg.mode = cmd_msg.VELOCITY_MODE
    cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()

    cmd_msg.velocity = list(qd)
    cmd_pub.publish(cmd_msg)
    time.sleep(delay)

set_vel_0()

In [71]:
set_vel_0()

In [84]:
xd=ee6-target
xd[3:6]=0
xd

array([-0.00324707,  0.00824831,  0.04589721,  0.        ,  0.        ,
        0.        ])

In [111]:
def goto_xyzrpy(target, delay=0.1):
    ee6=get_ee(ros_interface.latest_joint_states)
    xd=ee6-target

    alpha = .0001
    I = np.eye(q.shape[0])

    while np.linalg.norm(xd)>0.05:
        ee6=get_ee(ros_interface.latest_joint_states)
        xd=target-ee6 
        xd[3:6]=0

        J = robot.getJacobian('camera_link')
        
        qd = np.linalg.inv(J.T @ J + alpha * I) @ J.T @ xd

        cmd_msg = JointCommand()
        cmd_msg.names = robot.jointNames
        cmd_msg.mode = cmd_msg.VELOCITY_MODE
        cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()

        cmd_msg.velocity = list(qd)
        cmd_pub.publish(cmd_msg)
        time.sleep(delay)
    
    # stop
    cmd_msg = JointCommand()
    cmd_msg.names = robot.jointNames
    cmd_msg.mode = cmd_msg.VELOCITY_MODE
    cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
    cmd_msg.velocity =[0.0]*10
    cmd_pub.publish(cmd_msg)

In [110]:
goto_xyzrpy(ee6_0) 

In [109]:
goto_xyzrpy(ee6_n) 

### approach 2, jp jv

In [39]:
def set_vel_0():
    cmd_msg = JointCommand()
    cmd_msg.names = robot.jointNames
    cmd_msg.mode = cmd_msg.VELOCITY_MODE
    cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
    cmd_msg.velocity =[0.0]*10
    cmd_pub.publish(cmd_msg)

In [31]:
tpos=np.array(msgs[0].position)
cpos=np.array(ros_interface.latest_joint_states.position)
dpos=np.linalg.norm(tpos-cpos)
tpos, cpos, dpos

(array([-1.18468612e-01, -1.61511242e-01,  6.49876073e-02, -1.97506154e+00,
         1.37182248e+00, -1.40797150e+00, -1.97444212e+00,  3.79049480e-01,
         3.90922985e-14, -1.47179744e-05]),
 array([-1.18395224e-01, -1.61374107e-01,  6.50609583e-02,  3.05432391e+00,
         1.37168586e+00, -1.40805316e+00, -1.97435701e+00,  3.78911287e-01,
         7.54452412e-05, -8.24469498e-06]),
 5.029385456149283)

In [32]:
def goto_jointpos(target_jointpos, alpha=0.7, tol=0.1,  delay=0.1):
    cpos=np.array(ros_interface.latest_joint_states.position)
    dpos=np.linalg.norm(target_jointpos-cpos)

    while dpos>tol: 
        cpos=np.array(ros_interface.latest_joint_states.position)
        dpos=np.linalg.norm(target_jointpos-cpos)
 
        qd =(target_jointpos-cpos)*alpha

        cmd_msg = JointCommand()
        cmd_msg.names = robot.jointNames
        cmd_msg.mode = cmd_msg.VELOCITY_MODE
        cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
        cmd_msg.velocity = list(qd)
        cmd_pub.publish(cmd_msg)
        time.sleep(delay)

In [100]:
tpos=np.array(msgs[0].position)
goto_jointpos(tpos)

set_vel_0()

In [34]:
ros_interface.latest_joint_states.position

array('d', [-0.11844031512737274, -0.16137681901454926, 0.06501343101263046, -1.9362821578979492, 1.3717024326324463, -1.40804123878479, -1.9743659496307373, 0.3789162039756775, 4.6321235913637793e-07, -8.073320856283317e-08])

In [99]:
tpos=np.array(msgs[-1].position)
goto_jointpos(tpos)

set_vel_0()

In [36]:
ros_interface.robot.getJoints()

array([-1.91946417e-01, -1.61376774e-01,  2.71996558e-01, -1.95928919e+00,
        9.34154034e-01, -1.10973430e+00, -1.71904147e+00,  7.85836339e-01,
        4.94846745e-07,  0.00000000e+00])